construire un arbre hierarchique

event avec un carre plus heavy autour de la semaine

In [43]:
import numpy as np


basic_colors = {
    'red': (237, 28, 36),
    'pink': (255, 163, 177),
    'green': (34, 177, 76),
    'blue': (47, 54, 153),
    'cyan': (0, 183, 239),
    'yellow': (255, 194, 14),
    'orange': (255, 126, 0),
    'brown': (156, 90, 60),
    'purple': (111, 49, 152),
    'black': (0, 0, 0),
    'gray': (180, 180, 180),
    'white': (255, 255, 255)
}


In [44]:
import datetime

def get_week_number(year, month, day=1):
    date_obj = datetime.date(year, month, day)
    return date_obj.isocalendar()[1]



In [45]:
def get_rgb_value(basic_colors, color_string):
    if color_string.lower() in basic_colors:
        return basic_colors[color_string.lower()]
    elif color_string.lower().startswith('rgb'):
        rgb_values = color_string[color_string.index('(')+1:color_string.index(')')].split(',')
        return tuple(int(value.strip()) for value in rgb_values)
    else:
        return basic_colors['black']


def darken_rgb(rgb_tuple, darken_factor=0.5):
    darkened_rgb = tuple(int(max(0, val * (1 - darken_factor))) for val in rgb_tuple)
    return darkened_rgb


def lighten_rgb(rgb_tuple, lighten_factor=0.5):
    lightened_rgb = tuple(int(min(255, val + (255 - val) * lighten_factor)) for val in rgb_tuple)
    return lightened_rgb


def invert_rgb(rgb_tuple):
    inverted_rgb = tuple(255 - val for val in rgb_tuple)
    return inverted_rgb



In [46]:
import matplotlib.pyplot as plt
import math


def get_kernel(k_size=16):
    return np.full((k_size, k_size, 3), 255, dtype=np.uint8)


def get_square(x, y, size, color, k_size=16):
    kernel = get_kernel(k_size)
    kernel[x:x+size, y:y+size] = color
    kernel[x+1:x+size-1, y+1:y+size-1] = basic_colors['white']
    return kernel


def overlay(base, over):
    non_white_mask = ~np.all(over == [255, 255, 255], axis=-1)
    base[non_white_mask] = over[non_white_mask]
    return base


def get_journey(color):
    journey = get_square(0, 0, 6, darken_rgb(color), k_size=6)
    journey[1:6, 1:6] = color
    return journey


def count_journeys(cell):
    crop = cell[7:10, 7:10] #7:9
    crop = crop.reshape(-1, 3)
    pixels_strings = [','.join(map(str, pixel)) for pixel in crop]
    unique_pixels = list(set(pixels_strings))
    length = len(unique_pixels)
    return 0 if length == 1 and unique_pixels[0] == '255,255,255' else length


def add_journey(cell, color):
    count = count_journeys(cell)
    if count == 0:
        cell[5:11, 5:11] = get_journey(color)
    elif count == 1:
        cell[8:11, 5:11] = get_journey(color)[3:6, 0:6]
    elif count == 2:
        cell[5:8, 8:11] = get_journey(color)[0:3, 3:6]
    elif count == 3:
        cell[8:11, 8:11] = get_journey(color)[3:6, 3:6]
    return cell


def add_event(cell, color):
    cell = overlay(cell, get_square(3, 3, 11, darken_rgb(color)))
    cell = overlay(cell, get_square(2, 2, 11, color))
    return cell


def get_week():
    week = get_kernel()
    week = overlay(week, get_square(5, 5, 8, basic_colors['gray']))
    week = overlay(week, get_square(4, 4, 8, basic_colors['black']))
    return week


def get_year(w_gap=2):
    return np.full((16, (52*16 - 51*5 + 3*w_gap), 3), 255, dtype=np.uint8)


def get_life(length=100, w_gap=2, y_gap=2):
    return np.full(((length*16 - (length-1)*5 + int(math.ceil(length/5))*y_gap), (52*16 - 51*5 + 3*w_gap), 3), 255, dtype=np.uint8)


def add_week(year, week, week_number, w_gap=2):
    full_weel = get_year(w_gap)
    start = (week_number-1)*11 + ((week_number-1)//13)*w_gap
    full_weel[:, start:start+16] = week
    year = overlay(year, full_weel)
    return year


def add_year(life, year, year_number, life_len=100, w_gap=2, y_gap=2):
    full_year = get_life(life_len, w_gap, y_gap)
    start = (year_number)*11 + ((year_number)//5)*y_gap
    full_year[start:start+16, :] = year
    life = overlay(life, full_year)
    return life



In [47]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np

def add_text_to_array(array, y, x, text_size, text, font_path):
    """
    Add text to a numpy array.

    Args:
    - array (numpy.ndarray): The numpy array representing the image.
    - y (int): The y-coordinate for the text.
    - x (int): The x-coordinate for the text.
    - text_size (int): The size of the text.
    - text (str): The text to be added.
    - font_path (str): The path to the font file.

    Returns:
    - numpy.ndarray: The numpy array with the text added.
    """
    pil_image = Image.fromarray(array)
    font = ImageFont.truetype(font_path, size=text_size)
    draw = ImageDraw.Draw(pil_image)
    text_bbox = draw.textbbox((x, y), text, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]
    text_x = x - text_width // 2
    text_y = y - text_height // 2
    draw.text((text_x, text_y), text, fill=(0, 0, 0), font=font)
    modified_array = np.array(pil_image)
    return modified_array


def add_title(life, border, w_gap, font_path):
    life = add_text_to_array(life, 125, life.shape[1] // 2, 100, 'Life Calendar', font_path)
    life = add_text_to_array(life, 250, 10 * int(border + 3 + 13/2 * 11), 75, 'Winter', font_path)
    life = add_text_to_array(life, 250, 10 * int(border + w_gap + 3 + 13/2 * 11 * 3), 75, 'Spring', font_path)
    life = add_text_to_array(life, 250, 10 * int(border + 2*w_gap + 3 + 13/2 * 11 * 5), 75, 'Summer', font_path)
    life = add_text_to_array(life, 250, 10 * int(border + 3*w_gap + 3 + 13/2 * 11 * 7), 75, 'Autumn', font_path)
    return life


def add_year_number(life, year_start, year, y_gap, border, font_path):
    index = year - year_start
    height = 10 * int(border + index * 11 + index//5 * y_gap + 8)
    life = add_text_to_array(life, height, 250, 50, str(year), font_path)
    return life


def add_years(life, year_start, life_len, y_gap, border, font_path):
    for year in range(year_start, year_start + life_len):
        life = add_year_number(life, year_start, year, y_gap, border, font_path)
    return life



In [48]:
import json

with open('data/life.json', 'r') as f:
    data = json.load(f)

begin = int(data['begin'])
end = data['end'] if 'end' in data else begin + 100
life_len = end - begin
events = data['events']


events_dict = {}
journeys_dict = {}

# Process events
for event in data['events']:
    start_date = event['start']
    rgb_value = get_rgb_value(basic_colors, event['color'])
    start_year, start_month, start_day = map(int, start_date.split('-'))
    start_week_number = get_week_number(start_year, start_month, start_day)
    start_week_key = f"{start_year}-{start_week_number:02d}"
    if 'end' in event:
        end_date = event['end']
        end_year, end_month, end_day = map(int, end_date.split('-'))
        end_week_number = get_week_number(end_year, end_month, end_day)
        if start_year == end_year:
            for i in range(start_week_number, end_week_number + 1):
                key = f"{start_year}-{i:02d}"
                if key in events_dict:
                    journeys_dict[key].append(rgb_value)
                else:
                    journeys_dict[key] = [rgb_value]
        else:
            for year in range(start_year, end_year + 1):
                if year == start_year:
                    start = start_week_number
                    end = 52
                elif year == end_year:
                    start = 1
                    end = end_week_number
                else:
                    start = 1
                    end = 52
                for i in range(start, end + 1):
                    key = f"{year}-{i:02d}"
                    if key in journeys_dict:
                        journeys_dict[key].append(rgb_value)
                    else:
                        journeys_dict[key] = [rgb_value]
    else:
        if start_week_key in events_dict:
            events_dict[start_week_key].append(rgb_value)
        else:
            events_dict[start_week_key] = [rgb_value]

print("Events:")
print(events_dict)
print("\nJourneys:")
print(journeys_dict)


Events:
{'1998-08': [(237, 28, 36)], '2023-52': [(237, 28, 36)]}

Journeys:
{'2002-36': [(246, 180, 220)], '2002-37': [(246, 180, 220)], '2002-38': [(246, 180, 220)], '2002-39': [(246, 180, 220)], '2002-40': [(246, 180, 220)], '2002-41': [(246, 180, 220)], '2002-42': [(246, 180, 220)], '2002-43': [(246, 180, 220)], '2002-44': [(246, 180, 220)], '2002-45': [(246, 180, 220)], '2002-46': [(246, 180, 220)], '2002-47': [(246, 180, 220)], '2002-48': [(246, 180, 220)], '2002-49': [(246, 180, 220)], '2002-50': [(246, 180, 220)], '2002-51': [(246, 180, 220)], '2002-52': [(246, 180, 220)], '2003-01': [(246, 180, 220)], '2003-02': [(246, 180, 220)], '2003-03': [(246, 180, 220)], '2003-04': [(246, 180, 220)], '2003-05': [(246, 180, 220)], '2003-06': [(246, 180, 220)], '2003-07': [(246, 180, 220)], '2003-08': [(246, 180, 220)], '2003-09': [(246, 180, 220)], '2003-10': [(246, 180, 220)], '2003-11': [(246, 180, 220)], '2003-12': [(246, 180, 220)], '2003-13': [(246, 180, 220)], '2003-14': [(246, 180, 

In [49]:
font_path = './Bahnschrift-Font-Family/Bahnschrift.ttf'
w_gap = 2
y_gap = 2
border = 32

life = get_life(life_len, w_gap, y_gap)

for i in range(0, life_len):
    year = get_year(w_gap)
    for j in range(1, 53):
        week = get_week()
        week_key = f"{begin + i}-{j:02d}"
        if week_key in events_dict:
            for event in events_dict[week_key]:
                week = add_event(week, event)
        if week_key in journeys_dict:
            for journey in journeys_dict[week_key]:
                week = add_journey(week, journey)
        year = add_week(year, week, j, w_gap)
    life = add_year(life, year, i, life_len, w_gap, y_gap)


life = np.pad(life, ((border,border), (border,border), (0,0)), mode='constant', constant_values=255)
life = np.kron(life, np.ones((10,10,1), dtype=np.uint8))
life = add_title(life, border, w_gap, font_path)
life = add_years(life, begin, life_len, y_gap, border, font_path)


plt.imsave('life.png', life)